In [270]:
import pandas as pd
# Remove the limit on the number of columns displayed
pd.set_option('display.max_columns', None)

In [271]:
table_df = pd.read_csv("finaltable1.csv")

In [272]:
table_df.head()

,Unnamed: 0.1,Match ID,Season,Home Team,Away Team,Home Score,Away Score,Winner,Home Team ID,Away Team ID,Date,Time,Wk,Day,xG,Score,xG.1,Attendance,Venue,Referee,Away Win Odds,Home Win Odds,Draw Odds,StadiumID,Latitude,Longitude,Datetime,Temperature,Precipitation,H2H_Home_Wins_Last_4,H2H_Away_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A
0,0,497410,2024/2025,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,1,13,2024-08-16,19:00,1,Fri,2.4,1–0,0.4,73297.0,Old Trafford,Robert Jones,NaN,NaN,NaN,1,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,0.0,3,1,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,Manchester Utd,Fulham,2024-08-16_Manchester Utd_Fulham,Manchester Utd,Fulham,1.60,4.20,5.25
1,1,497411,2024/2025,Ipswich Town,Liverpool,0.0,2.0,AWAY_TEAM,2,20,2024-08-17,11:30,1,Sat,0.5,0–2,2.6,30014.0,Portman Road Stadium,Tim Robinson,NaN,NaN,NaN,2,52.0567,1.1450,2024-08-17T11:30:00,22.212500,0.0,0,0,0,2024-08-17_Ipswich Town_Liverpool,LIV,IPS,0.342601,3.929060,True,True,https://understat.com/match/26603,3.0,2.9562,3.929060,3.586459,8.739130,13.0,0.0,0.0258,0.342601,-3.586459,18.777778,2.0,2,1.0,5,2,Ipswich Town,Liverpool,2024-08-17_Ipswich Town_Liverpool,Ipswich Town,Liverpool,8.50,5.50,1.33
2,2,497412,2024/2025,Arsenal,Wolves,2.0,0.0,HOME_TEAM,3,19,2024-08-17,14:00,1,Sat,1.2,2–0,0.5,60261.0,Emirates Stadium,Jarred Gillett,NaN,NaN,NaN,3,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,0.0,0,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3,1.0,2,5,Arsenal,Wolves,2024-08-17_Arsenal_Wolves,Arsenal,Wolves,1.18,7.50,13.00
3,3,497413,2024/2025,Everton,Brighton,0.0,3.0,AWAY_TEAM,4,11,2024-08-17,14:00,1,Sat,0.5,0–3,1.4,39217.0,Goodison Park,Simon Hooper,NaN,NaN,NaN,4,53.4389,-2.9664,2024-08-17T14:00:00,17.587500,0.0,1,1,2,2024-08-17_Everton_Brighton,BRI,EVE,0.405325,1.790830,True,True,https://understat.com/match/26605,3.0,2.5287,1.790830,1.385505,7.916667,5.0,0.0,0.3078,0.405325,-1.385505,18.333333,4.0,4,1.0,2,2,Everton,Brighton,2024-08-17_Everton_Brighton,Everton,Brighton,2.63,3.30,2.63
4,4,497414,2024/2025,Newcastle Utd,Southampton,1.0,0.0,HOME_TEAM,5,15,2024-08-17,14:00,1,Sat,0.3,1–0,1.8,52196.0,St James' Park,Craig Pawson,NaN,NaN,NaN,5,54.9756,-1.6217,2024-08-17T14:00:00,18.622499,0.0,0,0,0,2024-08-17_Newcastle Utd_Southampton,SOU,NEW,0.433489,1.954830,True,True,https://understat.com/match/26606,0.0,2.5586,1.954830,1.521341,3.789474,13.0,3.0,0.2774,0.433489,-1.521341,16.250000,4.0,5,1.0,2,4,Newcastle Utd,Southampton,2024-08-17_Newcastle Utd_Southampton,Newcastle Utd,Southampton,1.36,5.25,8.00


In [273]:
len(table_df)

379

## Handling Missing Values

In [275]:
def check_missing_values():
    # Check for missing values in the dataset
    missing_values = table_df.isnull().sum()
    
    # Filter for columns with missing values
    missing_values = missing_values[missing_values > 0]
    
    # Display missing values
    print("Columns with Missing Values:")
    print(missing_values)


In [276]:
check_missing_values()

Columns with Missing Values:
Home Score               231
Away Score               231
Winner                   231
xG                       253
Score                    253
xG.1                     253
Attendance               257
Referee                  253
Away Win Odds            371
Home Win Odds            371
Draw Odds                371
Temperature              249
Precipitation            249
away_team_code            24
home_team_code            24
home_xg                  235
away_xg                  235
is_result                 24
has_data                  24
url                       24
away_points              231
away_expected_points     231
away_np_xg               231
away_np_xg_difference    231
away_ppda                231
away_deep_completions    231
home_points              231
home_expected_points     231
home_np_xg               231
home_np_xg_difference    231
home_ppda                231
home_deep_completions    231
HomeTeam                 230
AwayTeam      

Remove matches that have not been played..

In [278]:
table_df = table_df.dropna(subset=['Home Score', 'Away Score'])
# Reset the index after filtering
table_df.reset_index(drop=True, inplace=True)

In [279]:
check_missing_values()

Columns with Missing Values:
xG                22
Score             22
xG.1              22
Attendance        26
Referee           22
Away Win Odds    140
Home Win Odds    140
Draw Odds        140
Temperature       18
Precipitation     18
home_xg            4
away_xg            4
dtype: int64


In [280]:
# Fill missing 'xG' values with corresponding 'home_xg' values for the same row
table_df['xG'] = table_df.apply(
    lambda row: row['home_xg'] if pd.isnull(row['xG']) else row['xG'],
    axis=1
)
table_df['xG.1'] = table_df.apply(
    lambda row: row['away_xg'] if pd.isnull(row['xG.1']) else row['xG.1'],
    axis=1
)

# Verify that there are no more missing values in 'xG'
missing_xg_after = table_df['xG'].isnull().sum()
print(f"Remaining missing values in 'xG': {missing_xg_after}")
missing_xg_after = table_df['xG.1'].isnull().sum()
print(f"Remaining missing values in 'xG.1': {missing_xg_after}")


Remaining missing values in 'xG': 4
Remaining missing values in 'xG.1': 4


In [281]:
check_missing_values()

Columns with Missing Values:
xG                 4
Score             22
xG.1               4
Attendance        26
Referee           22
Away Win Odds    140
Home Win Odds    140
Draw Odds        140
Temperature       18
Precipitation     18
home_xg            4
away_xg            4
dtype: int64


Drop the score column because we already have home scores and away scores

In [283]:
# Drop the 'Score' column from the dataset
table_df.drop(columns=['Score','Precipitation'], inplace=True)

In [284]:
check_missing_values()

Columns with Missing Values:
xG                 4
xG.1               4
Attendance        26
Referee           22
Away Win Odds    140
Home Win Odds    140
Draw Odds        140
Temperature       18
home_xg            4
away_xg            4
dtype: int64


In [285]:
# Fill missing 'Attendance' values with the median attendance
median_attendance = table_df['Attendance'].median()
table_df['Attendance'].fillna(median_attendance, inplace=True)

# Verify there are no missing values in 'Attendance'
print(f"Remaining missing values in 'Attendance': {table_df['Attendance'].isnull().sum()}")


Remaining missing values in 'Attendance': 0


In [286]:
# Filter rows where 'Referee' is null
rows_with_missing_referee = table_df[table_df['Referee'].isnull()]
rows_with_missing_referee.head(7)

,Unnamed: 0.1,Match ID,Season,Home Team,Away Team,Home Score,Away Score,Winner,Home Team ID,Away Team ID,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,Away Win Odds,Home Win Odds,Draw Odds,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Wins_Last_4,H2H_Away_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A
126,126,497532,2024/2025,Chelsea,Aston Villa,3.0,0.0,HOME_TEAM,9,17,2024-12-01,13:30,13,Sun,1.949390,0.875443,35359.5,Stamford Bridge,NaN,NaN,NaN,NaN,9,51.4817,-0.1911,2024-12-01T13:30:00,12.771500,0,0,0,2024-12-01_Chelsea_Aston Villa,AVL,CHE,1.949390,0.875443,True,True,https://understat.com/match/26724,0.0,0.6308,0.875443,-1.073947,39.600000,4.0,3.0,2.1509,1.949390,1.073947,8.100000,4.0,127,13.0,3,4,Chelsea,Aston Villa,2024-12-01_Chelsea_Aston Villa,Chelsea,Aston Villa,1.70,4.1,4.50
127,127,497535,2024/2025,Manchester Utd,Everton,4.0,0.0,HOME_TEAM,1,4,2024-12-01,13:30,13,Sun,1.761610,0.766749,35359.5,Old Trafford,NaN,NaN,NaN,NaN,1,53.4631,-2.2914,2024-12-01T13:30:00,13.263000,0,0,0,2024-12-01_Manchester Utd_Everton,EVE,MUN,1.761610,0.766749,True,True,https://understat.com/match/26727,0.0,0.5986,0.766749,-0.994861,17.074074,3.0,3.0,2.1703,1.761610,0.994861,11.458333,10.0,128,13.0,2,3,Manchester Utd,Everton,2024-12-01_Manchester Utd_Everton,Manchester Utd,Everton,1.53,4.2,6.00
128,128,497537,2024/2025,Tottenham,Fulham,1.0,1.0,DRAW,16,13,2024-12-01,13:30,13,Sun,1.010860,1.572740,35359.5,Tottenham Hotspur Stadium,NaN,NaN,NaN,NaN,16,51.6047,-0.0664,2024-12-01T13:30:00,12.312500,0,0,0,2024-12-01_Tottenham_Fulham,FLH,TOT,1.010860,1.572740,True,True,https://understat.com/match/26729,1.0,1.8117,1.572740,0.561880,11.347826,7.0,1.0,0.9093,1.010860,-0.561880,19.307692,7.0,129,13.0,2,4,Tottenham,Fulham,2024-12-01_Tottenham_Fulham,Tottenham,Fulham,1.70,4.5,4.10
129,129,497534,2024/2025,Liverpool,Manchester City,2.0,0.0,HOME_TEAM,20,14,2024-12-01,16:00,13,Sun,3.700540,0.745015,35359.5,Anfield,NaN,NaN,NaN,NaN,20,53.4308,-2.9608,2024-12-01T16:00:00,11.907499,0,0,0,2024-12-01_Liverpool_Manchester City,MCI,LIV,3.700540,0.745015,True,True,https://understat.com/match/26726,0.0,0.0636,0.745015,-2.194355,15.687500,4.0,3.0,2.9016,2.939370,2.194355,17.352941,8.0,130,13.0,5,5,Liverpool,Manchester City,2024-12-01_Liverpool_Manchester City,Liverpool,Manchester City,2.10,3.6,3.30
130,130,497545,2024/2025,Ipswich Town,Crystal Palace,0.0,1.0,AWAY_TEAM,2,12,2024-12-03,19:30,14,Tue,0.471708,1.654220,35359.5,Portman Road Stadium,NaN,2.4,2.63,2.7,2,52.0567,1.1450,2024-12-03T19:30:00,NaN,0,0,0,2024-12-03_Ipswich Town_Crystal Palace,CRY,IPS,0.471708,1.654220,True,True,https://understat.com/match/26737,3.0,2.3167,1.654220,1.182512,13.565217,6.0,0.0,0.4642,0.471708,-1.182512,10.956522,2.0,131,14.0,2,2,Ipswich Town,Crystal Palace,2024-12-03_Ipswich Town_Crystal Palace,Ipswich Town,Crystal Palace,3.00,3.4,2.35
131,131,497546,2024/2025,Leicester City,West Ham,3.0,1.0,HOME_TEAM,10,7,2024-12-03,20:15,14,Tue,2.780610,2.877160,35359.5,King Power Stadium,NaN,NaN,NaN,NaN,10,52.6203,-1.1422,2024-12-03T20:15:00,NaN,2,1,1,2024-12-03_Leicester City_West Ham,WHU,LEI,2.780610,2.877160,True,True,https://understat.com/match/26738,0.0,1.4369,2.877160,0.096550,12.777778,14.0,3.0,1.3649,2.780610,-0.096550,10.068966,4.0,132,14.0,2,2,Leicester City,West Ham,2024-12-03_Leicester City_West Ham,Leicester City,West Ham,2.90,3.4,2.38
132,132,497543,2024/2025,Everton,Wolves,4.0,0.0,HOME_TEAM,4,19,2024-12-04,19:30,14,Wed,1.499040,0.847724,35359.5,Goodison Park,NaN,NaN,NaN,NaN,4,53.4389,-2.9664,2024-12-04T19:30:00,NaN,0,0,0,2024-12-04_Everton_Wolves,WOL,EVE,1.499040,0.847724,True,True,https://understat.com/m

In [287]:
# Fill missing 'Referee' values with the mode (most common referee)
mode_referee = table_df['Referee'].mode()[0]
table_df['Referee'].fillna(mode_referee, inplace=True)

# Verify there are no missing values in 'Referee'
print(f"Remaining missing values in 'Referee': {table_df['Referee'].isnull().sum()}")


Remaining missing values in 'Referee': 0


In [288]:
check_missing_values()

Columns with Missing Values:
xG                 4
xG.1               4
Away Win Odds    140
Home Win Odds    140
Draw Odds        140
Temperature       18
home_xg            4
away_xg            4
dtype: int64


In [289]:
from sklearn.impute import KNNImputer
import pandas as pd

# Load the dataset (assuming it's in table_df)
# Replace 'Temperature' and any other relevant columns for KNN imputation
columns_to_impute = ['Temperature']  # Include columns correlated with Temperature

# Select only the relevant columns for imputation
impute_df = table_df[columns_to_impute]

# Initialize the KNN imputer
knn_imputer = KNNImputer(n_neighbors=5)  # Use 5 neighbors by default

# Perform KNN imputation
imputed_data = knn_imputer.fit_transform(impute_df)

# Replace the original 'Temperature' column with the imputed values
table_df['Temperature'] = imputed_data[:, 0]

# Verify the imputation
print("Remaining missing values in 'Temperature':", table_df['Temperature'].isnull().sum())


Remaining missing values in 'Temperature': 0


In [290]:
check_missing_values()

Columns with Missing Values:
xG                 4
xG.1               4
Away Win Odds    140
Home Win Odds    140
Draw Odds        140
home_xg            4
away_xg            4
dtype: int64


In [291]:
columns_to_remove = ['Away Win Odds', 'Home Win Odds', 'Draw Odds']
table_df.drop(columns=columns_to_remove, inplace=True)


In [292]:
check_missing_values()

Columns with Missing Values:
xG         4
xG.1       4
home_xg    4
away_xg    4
dtype: int64


## Check for outliers

In [294]:
# Function to identify outliers based on IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)  # First quartile
    Q3 = df[column].quantile(0.75)  # Third quartile
    IQR = Q3 - Q1  # Interquartile range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Check for outliers in numerical columns
numerical_columns = table_df.select_dtypes(include=['float64', 'int64']).columns
outliers = {col: detect_outliers_iqr(table_df, col) for col in numerical_columns}

# Display the columns with outliers and their counts
outlier_counts = {col: len(outliers[col]) for col in outliers if len(outliers[col]) > 0}
print("Columns with outliers and their counts:")
print(outlier_counts)


Columns with outliers and their counts:
{'Home Score': 13, 'Away Score': 7, 'xG': 4, 'xG.1': 8, 'Temperature': 9, 'H2H_Home_Wins_Last_4': 9, 'H2H_Away_Wins_Last_4': 10, 'H2H_Draws_Last_4': 8, 'home_xg': 2, 'away_xg': 4, 'away_np_xg': 3, 'away_np_xg_difference': 4, 'away_ppda': 7, 'away_deep_completions': 6, 'home_np_xg': 2, 'home_np_xg_difference': 4, 'home_ppda': 8, 'home_deep_completions': 4, 'B365H': 13, 'B365D': 11, 'B365A': 9}


In [295]:
# Feature Engineering

In [296]:
table_df.head(11)

,Unnamed: 0.1,Match ID,Season,Home Team,Away Team,Home Score,Away Score,Winner,Home Team ID,Away Team ID,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Wins_Last_4,H2H_Away_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A
0,0,497410,2024/2025,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,1,13,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,3,1,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1,1.0,2,3,Manchester Utd,Fulham,2024-08-16_Manchester Utd_Fulham,Manchester Utd,Fulham,1.60,4.20,5.25
1,1,497411,2024/2025,Ipswich Town,Liverpool,0.0,2.0,AWAY_TEAM,2,20,2024-08-17,11:30,1,Sat,0.5,2.6,30014.0,Portman Road Stadium,Tim Robinson,2,52.0567,1.1450,2024-08-17T11:30:00,22.212500,0,0,0,2024-08-17_Ipswich Town_Liverpool,LIV,IPS,0.342601,3.929060,True,True,https://understat.com/match/26603,3.0,2.9562,3.929060,3.586459,8.739130,13.0,0.0,0.0258,0.342601,-3.586459,18.777778,2.0,2,1.0,5,2,Ipswich Town,Liverpool,2024-08-17_Ipswich Town_Liverpool,Ipswich Town,Liverpool,8.50,5.50,1.33
2,2,497412,2024/2025,Arsenal,Wolves,2.0,0.0,HOME_TEAM,3,19,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,0,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3,1.0,2,5,Arsenal,Wolves,2024-08-17_Arsenal_Wolves,Arsenal,Wolves,1.18,7.50,13.00
3,3,497413,2024/2025,Everton,Brighton,0.0,3.0,AWAY_TEAM,4,11,2024-08-17,14:00,1,Sat,0.5,1.4,39217.0,Goodison Park,Simon Hooper,4,53.4389,-2.9664,2024-08-17T14:00:00,17.587500,1,1,2,2024-08-17_Everton_Brighton,BRI,EVE,0.405325,1.790830,True,True,https://understat.com/match/26605,3.0,2.5287,1.790830,1.385505,7.916667,5.0,0.0,0.3078,0.405325,-1.385505,18.333333,4.0,4,1.0,2,2,Everton,Brighton,2024-08-17_Everton_Brighton,Everton,Brighton,2.63,3.30,2.63
4,4,497414,2024/2025,Newcastle Utd,Southampton,1.0,0.0,HOME_TEAM,5,15,2024-08-17,14:00,1,Sat,0.3,1.8,52196.0,St James' Park,Craig Pawson,5,54.9756,-1.6217,2024-08-17T14:00:00,18.622499,0,0,0,2024-08-17_Newcastle Utd_Southampton,SOU,NEW,0.433489,1.954830,True,True,https://understat.com/match/26606,0.0,2.5586,1.954830,1.521341,3.789474,13.0,3.0,0.2774,0.433489,-1.521341,16.250000,4.0,5,1.0,2,4,Newcastle Utd,Southampton,2024-08-17_Newcastle Utd_Southampton,Newcastle Utd,Southampton,1.36,5.25,8.00
5,5,497415,2024/2025,Nottingham Forest,Bournemouth,1.0,1.0,DRAW,6,18,2024-08-17,14:00,1,Sat,1.3,1.2,29763.0,The City Ground,Michael Oliver,6,52.9399,-1.1325,2024-08-17T14:00:00,19.121000,0,0,0,2024-08-17_Nottingham Forest_Bournemouth,BOU,NOT,1.244050,1.909150,True,True,https://understat.com/match/26607,1.0,1.8944,1.909150,0.665100,9.954545,4.0,1.0,0.8519,1.244050,-0.665100,8.653846,10.0,6,1.0,2,2,Nottingham Forest,Bournemouth,2024-08-17_Nottingham Forest_Bournemouth,Nottingham Forest,Bournemouth,2.45,3.50,2.80
6,6,497416,2024/2025,West Ham,Aston Villa,1.0,2.0,AWAY_TEAM,7,17,2024-08-17,16:30,1,Sat,2.3,2.0,62463.0,London Stadium,Tony Harrington,7,51.5386,-0.0164,2024-08-17T16:30:00,21.945499,0,0,0,2024-08-17_West Ham_Aston Villa,AVL,WHU,2.804700,2.670300,True,True,https://understat.com/match/26608,3.0,1.2989,2.670300,0.626770,11.304348,8.0,0.0,1.4690,2.043530,-0.626770,9.333333,5.0,7,1.0,3,2,West Ham,Aston Villa,2024-08-17_West Ham_Aston Villa,West Ham,Aston Vil

In [297]:
#Feature Engineering

### Home PPDA Average

In [300]:
# # Calculate home_avg_ppda and away_avg_ppda
# table_df['home_avg_ppda'] = table_df['home_ppda'] / table_df['Home Played']
# table_df['away_avg_ppda'] = table_df['away_ppda'] / table_df['Away Played']


In [ ]:
# table_df[['Match ID','Home Team','Away Team','Home Played','Away Played','home_ppda','away_ppda','home_avg_ppda','away_avg_ppda']].head()